In [1]:
import sys
sys.path.append("../..")

import os
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time

import pysingfel as ps
import pysingfel.gpu as pg

In [2]:
# Create a particle object
particle = ps.Particle()
particle.read_pdb('../input/3iyf.pdb', ff='WK')

# Load beam
beam = ps.Beam('../input/exp_chuck.beam') 

# Load and initialize the detector
det = ps.PnccdDetector(geom='../lcls/amo86615/PNCCD::CalibV1/Camp.0:pnCCD.1/geometry/0-end.data', 
                       beam=beam)

# Calculate the 3D diffraction volume

In [3]:
mesh_length = 128
mesh, voxel_length = det.get_reciprocal_mesh(voxel_number_1d = mesh_length)
volume = pg.calculate_diffraction_pattern_gpu(mesh, particle, return_type = 'intensity')

# Take 2 slices

In [4]:
# Generate n images uniformly over 4-sphere
n = 2
orientations = np.array([
    [1., 1., 0., 0.],
    [-1., 0., 1., 0.]]
) / np.sqrt(2)

slices = ps.geometry.take_n_slice(
    pattern_shape = det.pedestal.shape,
    pixel_momentum = det.pixel_position_reciprocal,
    volume = volume,
    voxel_length = voxel_length,
    orientations = orientations)

vshape = volume.shape
ishape =(n, 4, 512, 512)

with h5.File('imStack-test.hdf5','w') as f:
    f.create_dataset('volume', shape=vshape, maxshape=vshape, data=volume, dtype=np.float64)
    f.create_dataset('imUniform', shape=ishape, maxshape=ishape, data=slices, dtype=np.float64)
    f.create_dataset('imOrientations', orientations.shape, data=orientations, dtype=np.float64)

with h5.File('imStack-test.hdf5','r') as f:
    volume_in = f['volume'][:]
    slices_in = f['imUniform'][:]
    orientations_in = f['imOrientations'][:]

Finishing constructing 2 patterns in 1.970150 seconds


# >>> Move the file in the test directory manually <<<
This is not automated to avoid changing it by mistake

# Check for consistency

In [5]:
assert np.allclose(volume, volume_in)
assert np.allclose(slices, slices_in)
assert np.allclose(orientations, orientations_in)

In [6]:
slices_rec = ps.geometry.take_n_slice(
    pattern_shape = det.pedestal.shape,
    pixel_momentum = det.pixel_position_reciprocal,
    volume = volume_in,
    voxel_length = voxel_length,
    orientations = orientations_in)

Finishing constructing 2 patterns in 1.741647 seconds


In [7]:
# Note: This does not work if orientations is stored as float32
assert np.allclose(slices_in, slices_rec)